# THIS NOTEBOOK HAS MINIMUM COMMENTING/MARKDOWN BECAUSE I HAD TO RECREATE IT THIS MORNING

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
from datetime import datetime

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
a = pd.read_csv('./Data/final_hilton.csv')
a = a.drop(['Unnamed: 0'], axis=1)
b = pd.read_csv('./Data/final_hyatt.csv')
b = b.drop(['Unnamed: 0'], axis=1)
c = pd.read_csv('./Data/final_ihg.csv')
c = c.drop(['Unnamed: 0'], axis=1)
d = pd.read_csv('./Data/final_marriott.csv')
d = d.drop(['Unnamed: 0'], axis=1)
e = pd.read_csv('./Data/final_radisson.csv')
e = e.drop(['Unnamed: 0'], axis=1)
f = pd.read_csv('./Data/final_wyndham.csv')
f = f.drop(['Unnamed: 0'], axis=1)

In [3]:
a.head(1)

,name,description,streetAddress,addressLocality,addressRegion,addressCountry,postalCode,url,latitude,longitude,rating,brand
0,Hampton Inn Alexander City,"We’re right off Highway 280, 25 minutes away f...",1551 Elkahatchee Road,Alexander City,AL,US,35010,https://hamptoninn3.hilton.com/en/hotels/alaba...,32.927516,-85.966271,4,Hampton Inn


In [4]:
b.head(1)

,name,url,street,city,state,postal,description,rating
0,Hyatt Place Birmingham/Hoover,https://www.hyatt.com/en-US/hotel/alabama/hyat...,2980 John Hawkins Parkway,Hoover,ALABAMA,35244,Relax poolside or enjoy a refreshing dip in o...,4.5 Stars


In [5]:
c.head(1)

,name,description,url,street,locality,region,postalcode,address3,rating,brand
0,Holiday Inn Express & Suites Alabaster,"The Holiday Inn Express & Suites, Alabaster in...",https://www.ihg.com/holidayinnexpress/hotels/u...,"1000 Balmoral Drive ,","Alabaster,",Alabama,35007,NaN,4.7,Holiday Inn Express


In [6]:
d.head(1)

,brand,name,url,street,locality,state,postalcode,coordinate1,coordinate2,coordinate3,coordinate4,description,overall_rating,cleanliness_rating,service_rating
0,AC Hotel,AC Hotel Huntsville Downtown,https://www.marriott.com/HSVAR,435 Williams Avenue SW,Huntsville,ALABAMA,35801,-86.58764,NaN,NaN,NaN,[ Discover something new at the AC Hotel Hunts...,NaN,4.7,4.5


In [7]:
e.head(1)

,name,url,street,locality,state,postal,description,rating
0,"Country Inn & Suites by Radisson, Dothan, AL",https://www.radissonhotels.com/en-us/hotels/co...,3465 Ross Clark Circle,Dothan,Alabama,36303,"Conveniently located on travel routes, the Co...",https://www.tripadvisor.com/img/cdsi/img2/rati...


In [8]:
f.head(1)

,name,url,street,city,state,postal,description,rating
0,Microtel Inn & Suites by Wyndham Albertville,https://www.wyndhamhotels.com//microtel/albert...,220 Highway 75 North,Albertville,AL,35950,Stop Off in Albertville Pet-friendly hotel...,4.0


In [9]:
print(a.columns)
print('='*50)
print(b.columns)
print('='*50)
print(c.columns)
print('='*50)
print(d.columns)
print('='*50)
print(e.columns)
print('='*50)
print(f.columns)
print('='*50)

Index(['name', 'description', 'streetAddress', 'addressLocality',
       'addressRegion', 'addressCountry', 'postalCode', 'url', 'latitude',
       'longitude', 'rating', 'brand'],
      dtype='object')
Index(['name', 'url', 'street', 'city', 'state', 'postal', 'description',
       'rating'],
      dtype='object')
Index(['name', 'description', 'url', 'street', 'locality', 'region',
       'postalcode', 'address3', 'rating', 'brand'],
      dtype='object')
Index(['brand', 'name', 'url', 'street', 'locality', 'state', 'postalcode',
       'coordinate1', 'coordinate2', 'coordinate3', 'coordinate4',
       'description', 'overall_rating', 'cleanliness_rating',
       'service_rating'],
      dtype='object')
Index(['name', 'url', 'street', 'locality', 'state', 'postal', 'description',
       'rating'],
      dtype='object')
Index(['name', 'url', 'street', 'city', 'state', 'postal', 'description',
       'rating'],
      dtype='object')


In [10]:
e = e.rename(columns={"locality": "city"})

In [11]:
d = d.drop(['coordinate1', 'coordinate2', 'coordinate3', 'coordinate4', 'overall_rating', 'cleanliness_rating'], axis=1)

In [12]:
d = d.rename(columns={'service_rating' : 'rating', 'postalcode' : 'postal', 'locality' : 'city'})

In [13]:
c = c.rename(columns={'postalcode' : 'postal', 'region' : 'state', 'locality' : 'city'})

In [14]:
c = c.drop(['address3'], axis=1)

In [15]:
a = a.rename(columns={'streetAddress' : 'street', 'addressLocality' : 'city',
       'addressRegion' : 'state', 'postalCode' : 'postal'})

In [16]:
a = a.drop(['addressCountry', 'latitude', 'longitude'], axis=1)

In [17]:
b['brand'] = ''

In [18]:
for index, row in b.iterrows():
    if 'Hyatt Place' in row['name']:
        b['brand'][index] = 'Hyatt Place'
    elif 'Hyatt House' in row['name']:
        b['brand'][index] = 'Houte House'
    elif 'Hyatt Regency' in row['name']:
        b['brand'][index] = 'Hyatt Regency'
    elif 'Grand Hyatt' in row['name']:
        b['brand'][index] = 'Grand Hyatt'
    elif 'Park Hyatt' in row['name']:
        b['brand'][index] = 'Park Hyatt'
    elif 'Andaz' in row['name']:
        b['brand'][index] = 'Andaz'
    elif 'Thompson' in row['name']:
        b['brand'][index] = 'Thompson'
    elif 'Centric' in row['name']:
        b['brand'][index] = 'Hyatt Centric'
    elif 'miraval' in row['url']:
        b['brand'][index] = 'MiRaval'
    elif 'jdv-' in row['url']:
        b['brand'][index] = 'Joie De Vivre'
    elif 'hyatt-' in row['url']:
        b['brand'][index] = 'Hyatt'
#     elif 'Canopy' in row['name']:
#         b['brand'][index] = 'Canopy'
#     elif 'Curio' in row['name']:
#         b['brand'][index] = 'Curio'
#     elif 'Hilton Grand' in row['name']:
#         b['brand'][index] = 'Hilton Grand Vacations'
#     elif 'Tapestry' in row['name']:
#         b['brand'][index] = 'Tapestry'
#     elif 'Motto' in row['name']:
#         b['brand'][index] = 'Motto'
    else:
        b['brand'][index] = 'Destination or Unbound'

In [19]:
b['brand'].value_counts()

Hyatt Place               323
Houte House               100
Hyatt Regency              97
Destination or Unbound     97
Hyatt Centric              22
Hyatt                      20
Joie De Vivre              16
Grand Hyatt                13
Andaz                       9
Thompson                    7
Park Hyatt                  6
MiRaval                     3
Name: brand, dtype: int64

In [20]:
for x in b[(b['brand']=='N/A')]['url']:
    print(x)

In [21]:
e['brand'] = ''

In [22]:
for index, row in e.iterrows():
    if 'Country Inn' in row['name']:
        e['brand'][index] = 'Country Inn & Suites'
    elif 'Park Inn' in row['name']:
        e['brand'][index] = 'Park Inn'
    elif 'Park Plaza' in row['name']:
        e['brand'][index] = 'Park Plaza'
#     elif 'Doubletree' in row['name']:
#         e['brand'][index] = 'Doubletree'
#     elif 'DoubleTree' in row['name']:
#         hilton['brand'][index] = 'Doubletree'
#     elif 'Embassy' in row['name']:
#         hilton['brand'][index] = 'Embassy Suites'
#     elif 'Homewood Suites' in row['name']:
#         hilton['brand'][index] = 'Homewood Suites'
#     elif 'Home2 Suites' in row['name']:
#         hilton['brand'][index] = 'Home2'
#     elif 'Home2 by' in row['name']:
#         hilton['brand'][index] = 'Home2'
#     elif 'Conrad' in row['name']:
#         hilton['brand'][index] = 'Conrad'
#     elif 'Waldorf' in row['name']:
#         hilton['brand'][index] = 'Waldorf Astoria'
#     elif 'Canopy' in row['name']:
#         hilton['brand'][index] = 'Canopy'
#     elif 'Curio' in row['name']:
#         hilton['brand'][index] = 'Curio'
#     elif 'Hilton Grand' in row['name']:
#         hilton['brand'][index] = 'Hilton Grand Vacations'
#     elif 'Tapestry' in row['name']:
#         hilton['brand'][index] = 'Tapestry'
#     elif 'Motto' in row['name']:
#         hilton['brand'][index] = 'Motto'
    else:
        e['brand'][index] = 'Radisson'

In [23]:
e['brand'].value_counts()

Country Inn & Suites    446
Radisson                 81
Park Inn                  6
Name: brand, dtype: int64

In [24]:
f['brand'] = ''

In [25]:
for index, row in f.iterrows():
    if 'Hawthorn' in row['name']:
        f['brand'][index] = 'Hawthorn Suites'
    elif 'Travelodge' in row['name']:
        f['brand'][index] = 'Travelodge'
    elif 'Howard Johnson' in row['name']:
        f['brand'][index] = 'Howard Johnson'
    elif 'Super 8' in row['name']:
        f['brand'][index] = 'Super 8'
    elif 'Days Inn' in row['name']:
        f['brand'][index] = 'Days Inn'
    elif 'Microtel' in row['name']:
        f['brand'][index] = 'Microtel'
    elif 'Ramada' in row['name']:
        f['brand'][index] = 'Ramada'
    elif 'AmericInn' in row['name']:
        f['brand'][index] = 'AmericInn'
    elif 'Wyndham Garden' in row['name']:
        f['brand'][index] = 'Wyndham Garden'
    elif 'Wingate' in row['name']:
        f['brand'][index] = 'Wingate'
    elif 'La Quinta' in row['name']:
        f['brand'][index] = 'La Quinta'
    elif 'Trademark' in row['name']:
        f['brand'][index] = 'Trademark'
    elif 'Dazzler' in row['name']:
        f['brand'][index] = 'Dazzler'
    elif 'Esplendor' in row['name']:
        f['brand'][index] = 'Esplendor'
    elif 'Tryp' in row['name']:
        f['brand'][index] = 'TRYP'
    elif 'TRYP' in row['name']:
        f['brand'][index] = 'TRYP'
    elif 'Dolce' in row['name']:
        f['brand'][index] = 'Dolce'
    elif 'Wyndham Grand' in row['name']:
        f['brand'][index] = 'Wyndham Grand'
    elif 'Club Wyndham' in row['name']:
        f['brand'][index] = 'Club Wyndham'
    elif 'Worldmark' in row['name']:
        f['brand'][index] = 'Worldmark'
    elif 'Margaritaville' in row['name']:
        f['brand'][index] = 'Margaritaville'
    elif 'Caesars' in row['name']:
        f['brand'][index] = 'Caesars'
    elif 'Harrahs' in row['name']:
        f['brand'][index] = 'Harrahs'
    elif 'Horseshoe' in row['name']:
        f['brand'][index] = 'Horseshoe'
    elif 'Cromwell' in row['name']:
        f['brand'][index] = 'The Cromwell'
    elif 'Bally' in row['name']:
        f['brand'][index] = "Bally's"
    elif 'Flamingo' in row['name']:
        f['brand'][index] = 'Flamingo'
    elif 'Linq' in row['name']:
        f['brand'][index] = 'The LINQ'
    elif 'NOBU' in row['name']:
        f['brand'][index] = 'NOBU'
    elif 'Paris' in row['name']:
        f['brand'][index] = "Paris"
    elif 'Planet Hollywood' in row['name']:
        f['brand'][index] = 'Planet Hollywood'
    elif 'Rio' in row['name']:
        f['brand'][index] = 'Rio'
    elif 'Baymont' in row['name']:
        f['brand'][index] = 'Baymont'
    elif 'Days' in row['name']:
        f['brand'][index] = 'Days Inn'
    elif 'wyndham-vacations' in row['url']:
        f['brand'][index] = 'Wyndham Vacations'
#     elif 'Days' in row['name']:
#         f['brand'][index] = 'Days Inn'
    else:
        f['brand'][index] = 'Wyndham'

In [26]:
f['brand'].value_counts()

Super 8              1474
Days Inn             1375
La Quinta             888
Baymont               501
Travelodge            335
Ramada                316
Microtel              291
AmericInn             199
Wingate               163
Howard Johnson        159
Hawthorn Suites       101
Wyndham Vacations      81
Wyndham Garden         67
Club Wyndham           67
Wyndham                49
Trademark              41
Wyndham Grand          18
TRYP                    9
Caesars                 4
Paris                   2
Dolce                   2
Horseshoe               2
Planet Hollywood        1
The Cromwell            1
Bally's                 1
Flamingo                1
Name: brand, dtype: int64

In [27]:
for x in f[(f['brand']=='N/A')]['url']:
    print(x)
    print

In [28]:
a['rewards'] = 'Hilton'
b['rewards'] = 'Hyatt'
c['rewards'] = 'IHG'
d['rewards'] = 'Marriott'
e['rewards'] = 'Radisson'
f['rewards'] = 'Wyndham'

In [29]:
a.head(1)

,name,description,street,city,state,postal,url,rating,brand,rewards
0,Hampton Inn Alexander City,"We’re right off Highway 280, 25 minutes away f...",1551 Elkahatchee Road,Alexander City,AL,35010,https://hamptoninn3.hilton.com/en/hotels/alaba...,4,Hampton Inn,Hilton


In [30]:
a= a[['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city', 'state', 'postal', 'url']]

In [31]:
a.head(1)

,rewards,brand,name,rating,description,street,city,state,postal,url
0,Hilton,Hampton Inn,Hampton Inn Alexander City,4,"We’re right off Highway 280, 25 minutes away f...",1551 Elkahatchee Road,Alexander City,AL,35010,https://hamptoninn3.hilton.com/en/hotels/alaba...


In [32]:
b.head(1)

,name,url,street,city,state,postal,description,rating,brand,rewards
0,Hyatt Place Birmingham/Hoover,https://www.hyatt.com/en-US/hotel/alabama/hyat...,2980 John Hawkins Parkway,Hoover,ALABAMA,35244,Relax poolside or enjoy a refreshing dip in o...,4.5 Stars,Hyatt Place,Hyatt


In [33]:
b= b[['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city', 'state', 'postal', 'url']]

In [34]:
b.head(1)

,rewards,brand,name,rating,description,street,city,state,postal,url
0,Hyatt,Hyatt Place,Hyatt Place Birmingham/Hoover,4.5 Stars,Relax poolside or enjoy a refreshing dip in o...,2980 John Hawkins Parkway,Hoover,ALABAMA,35244,https://www.hyatt.com/en-US/hotel/alabama/hyat...


In [35]:
b.head(1)

,rewards,brand,name,rating,description,street,city,state,postal,url
0,Hyatt,Hyatt Place,Hyatt Place Birmingham/Hoover,4.5 Stars,Relax poolside or enjoy a refreshing dip in o...,2980 John Hawkins Parkway,Hoover,ALABAMA,35244,https://www.hyatt.com/en-US/hotel/alabama/hyat...


In [36]:
c.head(1)

,name,description,url,street,city,state,postal,rating,brand,rewards
0,Holiday Inn Express & Suites Alabaster,"The Holiday Inn Express & Suites, Alabaster in...",https://www.ihg.com/holidayinnexpress/hotels/u...,"1000 Balmoral Drive ,","Alabaster,",Alabama,35007,4.7,Holiday Inn Express,IHG


In [37]:
c= c[['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city', 'state', 'postal', 'url']]

In [38]:
c.head(1)

,rewards,brand,name,rating,description,street,city,state,postal,url
0,IHG,Holiday Inn Express,Holiday Inn Express & Suites Alabaster,4.7,"The Holiday Inn Express & Suites, Alabaster in...","1000 Balmoral Drive ,","Alabaster,",Alabama,35007,https://www.ihg.com/holidayinnexpress/hotels/u...


In [39]:
d.head(1)

,brand,name,url,street,city,state,postal,description,rating,rewards
0,AC Hotel,AC Hotel Huntsville Downtown,https://www.marriott.com/HSVAR,435 Williams Avenue SW,Huntsville,ALABAMA,35801,[ Discover something new at the AC Hotel Hunts...,4.5,Marriott


In [40]:
d= d[['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city', 'state', 'postal', 'url']]

In [41]:
d.head(1)

,rewards,brand,name,rating,description,street,city,state,postal,url
0,Marriott,AC Hotel,AC Hotel Huntsville Downtown,4.5,[ Discover something new at the AC Hotel Hunts...,435 Williams Avenue SW,Huntsville,ALABAMA,35801,https://www.marriott.com/HSVAR


In [42]:
e.head(1)

,name,url,street,city,state,postal,description,rating,brand,rewards
0,"Country Inn & Suites by Radisson, Dothan, AL",https://www.radissonhotels.com/en-us/hotels/co...,3465 Ross Clark Circle,Dothan,Alabama,36303,"Conveniently located on travel routes, the Co...",https://www.tripadvisor.com/img/cdsi/img2/rati...,Country Inn & Suites,Radisson


In [43]:
e= e[['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city', 'state', 'postal', 'url']]

In [44]:
e.head(1)

,rewards,brand,name,rating,description,street,city,state,postal,url
0,Radisson,Country Inn & Suites,"Country Inn & Suites by Radisson, Dothan, AL",https://www.tripadvisor.com/img/cdsi/img2/rati...,"Conveniently located on travel routes, the Co...",3465 Ross Clark Circle,Dothan,Alabama,36303,https://www.radissonhotels.com/en-us/hotels/co...


In [45]:
f.head(1)

,name,url,street,city,state,postal,description,rating,brand,rewards
0,Microtel Inn & Suites by Wyndham Albertville,https://www.wyndhamhotels.com//microtel/albert...,220 Highway 75 North,Albertville,AL,35950,Stop Off in Albertville Pet-friendly hotel...,4.0,Microtel,Wyndham


In [46]:
f= f[['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city', 'state', 'postal', 'url']]

In [47]:
f.head()

,rewards,brand,name,rating,description,street,city,state,postal,url
0,Wyndham,Microtel,Microtel Inn & Suites by Wyndham Albertville,4.0,Stop Off in Albertville Pet-friendly hotel...,220 Highway 75 North,Albertville,AL,35950,https://www.wyndhamhotels.com//microtel/albert...
1,Wyndham,Days Inn,Days Inn by Wyndham Alexander City,4.0,Rest Easy in Alexander City Near Lake Mart...,3146 Highway 280,Alexander City,AL,35010,https://www.wyndhamhotels.com//days-inn/alexan...
2,Wyndham,Super 8,Super 8 by Wyndham Alexander City,4.0,Enjoy Alexander City Pet-friendly location...,4335 Hwy 280,Alexander City,AL,35010,https://www.wyndhamhotels.com//super-8/alexand...
3,Wyndham,Days Inn,Days Inn by Wyndham Andalusia,3.0,Stop Off in Andalusia Across from LBW Comm...,1604 Dr MLK Jr Expressway,Andalusia,AL,36420,https://www.wyndhamhotels.com//days-inn/andalu...
4,Wyndham,Days Inn,Days Inn & Suites by Wyndham Athens Alabama,4.0,Explore the Antebellum South Off I-65 near...,1322 Highway 72 East,Athens,AL,35611-4406,https://www.wyndhamhotels.com//days-inn/athens...


In [48]:
a.shape

(4890, 10)

In [49]:
b.shape

(713, 10)

In [50]:
c.shape

(3970, 10)

In [51]:
d.shape

(5191, 10)

In [52]:
e.shape

(533, 10)

In [53]:
f.shape

(6148, 10)

In [54]:
a.columns

Index(['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city',
       'state', 'postal', 'url'],
      dtype='object')

In [55]:
b.columns

Index(['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city',
       'state', 'postal', 'url'],
      dtype='object')

In [56]:
c.columns

Index(['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city',
       'state', 'postal', 'url'],
      dtype='object')

In [57]:
d.columns

Index(['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city',
       'state', 'postal', 'url'],
      dtype='object')

In [58]:
e.columns

Index(['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city',
       'state', 'postal', 'url'],
      dtype='object')

In [59]:
f.columns

Index(['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city',
       'state', 'postal', 'url'],
      dtype='object')

In [60]:
for x in a['state']:
    print(x)

AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
AL
Alabama
AK
AK
AK
AK
AK
AK
AK
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AZ
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
AR
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA
CA

In [61]:
a['state'].value_counts()

TX                                             474
FL                                             360
CA                                             326
GA                                             210
NC                                             204
OH                                             187
NY                                             183
PA                                             181
VA                                             176
TN                                             149
IL                                             129
SC                                             121
AL                                             120
IN                                             110
CO                                             104
AZ                                              97
MI                                              94
MD                                              88
OK                                              85
NJ                             

In [62]:
merged_hotels = pd.concat([a, b, c, d, e, f], ignore_index=True)

In [63]:
merged_hotels.shape

(21445, 10)

In [64]:
merged_hotels['state'].value_counts()

TX                                              1132
CA                                               748
FL                                               680
TEXAS                                            612
CALIFORNIA                                       584
Texas                                            549
FLORIDA                                          497
GA                                               492
NC                                               394
NY                                               349
OH                                               345
VA                                               344
TN                                               330
IL                                               328
PA                                               318
Florida                                          298
California                                       291
SC                                               256
MI                                            

In [65]:
for x in merged_hotels['state']:
    try:
        x = x.upper()
    except:
        pass

In [66]:
newstate = []
for x in merged_hotels['state']:
    try:
        x = x.upper()
        newstate.append(x)
    except:
        newstate.append(x)

In [67]:
len(newstate)

21445

In [68]:
merged_hotels['state'] = newstate

In [69]:
merged_hotels.head(5)

,rewards,brand,name,rating,description,street,city,state,postal,url
0,Hilton,Hampton Inn,Hampton Inn Alexander City,4,"We’re right off Highway 280, 25 minutes away f...",1551 Elkahatchee Road,Alexander City,AL,35010,https://hamptoninn3.hilton.com/en/hotels/alaba...
1,Hilton,Hampton Inn,Hampton Inn Wetumpka,5,"We’re on the banks of the Coosa River, a short...",350 South Main Street,Wetumpka,AL,36092,https://hamptoninn3.hilton.com/en/hotels/alaba...
2,Hilton,Hampton Inn,Hampton Inn Auburn,4,"We're off I-85, under 10 minutes from Chewacla...",2430 S. College St.,Auburn,AL,36832,https://hamptoninn3.hilton.com/en/hotels/alaba...
3,Hilton,Tru,Tru by Hilton Auburn,4,"Make this Tru by Hilton your own. We’ll help, ...",2411 W Pace Blvd,Auburn,AL,36830,https://tru3.hilton.com/en/hotels/alabama/tru-...
4,Hilton,Hilton Garden Inn,Hilton Garden Inn Auburn/Opelika,4,"Our hotel is just off I-85, minutes away from ...",2555 Hilton Garden Drive,Auburn,AL,36830,https://hiltongardeninn3.hilton.com/en/hotels/...


In [70]:
merged_hotels['state'].value_counts()

TEXAS                                           1161
TX                                              1132
CALIFORNIA                                       875
FLORIDA                                          795
CA                                               748
FL                                               680
GA                                               492
GEORGIA                                          416
NEW YORK                                         399
NC                                               394
NORTH CAROLINA                                   382
VIRGINIA                                         368
PENNSYLVANIA                                     362
OHIO                                             351
NY                                               349
OH                                               345
VA                                               344
ILLINOIS                                         344
TN                                            

In [71]:
statereplace = {'ALABAMA' : 'AL',
'ALASKA' : 'AK',
'ARIZONA' : 'AZ',
'ARKANSAS' : 'AR',
'CALIFORNIA' : 'CA',
'COLORADO' : 'CO',
'CONNECTICUT' : 'CT',
'DELAWARE' : 'DE',
'DISTRICT OF COLUMBIA' : 'DC',
'FLORIDA' : 'FL',
'GEORGIA' : 'GA',
'HAWAII' : 'HI',
'IDAHO' : 'ID',
'ILLINOIS' : 'IL',
'INDIANA' : 'IN',
'IOWA' : 'IA',
'KANSAS' : 'KS',
'KENTUCKY' : 'KY',
'LOUISIANA' : 'LA',
'MAINE' : 'ME',
'MARYLAND' : 'MD',
'MASSACHUSETTS' : 'MA',
'MICHIGAN' : 'MI', 
'MINNESOTA' : 'MN',
'MISSISSIPPI' : 'MS',
'MISSOURI' : 'MO',
'MONTANA' : 'MT',
'NEBRASKA' : 'NE',
'NEVADA' : 'NV',
'NEW HAMPSHIRE' : 'NH',
'NEW JERSEY' : 'NJ',
'NEW MEXICO' : 'NM', 
'NEW YORK' : 'NY',
'NORTH CAROLINA' : 'NC',
'NORTH DAKOTA' : 'ND',
'OHIO' : 'OH',
'OKLAHOMA' : 'OK',
'OREGON' : 'OR',
'PENNSYLVANIA' : 'PA',
'RHODE ISLAND' : 'RI',
'SOUTH CAROLINA' : 'SC',
'SOUTH DAKOTA' : 'SD',
'TENNESSEE' : 'TN',
'TEXAS' : 'TX',
'UTAH' : 'UT',
'VERMONT' : 'VT',
'VIRGINIA' : 'VA',
'WASHINGTON' : 'WA',
'WEST VIRGINIA' : 'WV',
'WISCONSIN' : 'WI',
'WYOMING' : 'WY'}

In [72]:
merged_hotels['state'] = merged_hotels['state'].replace(statereplace)


In [73]:
merged_hotels['state'] = merged_hotels['state'].str.strip()

In [74]:
merged_hotels['state'].value_counts()

TX                                             2297
CA                                             1649
FL                                             1495
GA                                              912
NC                                              782
NY                                              752
VA                                              714
OH                                              698
PA                                              686
IL                                              680
TN                                              602
MI                                              548
CO                                              514
IN                                              487
SC                                              479
AZ                                              456
WI                                              425
MO                                              401
AL                                              398
MN          

In [75]:
statereplace2 = {'GEORGIA31904' : 'GA', 'NEW YORK 1483' : 'NY', '<SPAN CLASS="PROPERTY-MISSOURI</SPAN>' : 'MO', 
                '<SPAN CLASS="PROPERTY-ARIZONA</SPAN>' : 'AZ', 'CALIFORNIA9' : 'CA', 'ILLINOIS61704' : 'IL', 
                '<SPAN CLASS="PROPERTY-MAINE</SPAN>' : 'ME', '<SPAN CLASS="PROPERTY-NORTH CAROLINA</SPAN>' : 'NC',
                '<SPAN CLASS="PROPERTY-KANSAS</SPAN>' : 'KS', '<SPAN CLASS="PROPERTY-ALABAMA</SPAN>' : 'AL', 
                '<SPAN CLASS="PROPERTY-WISCONSIN</SPAN>' : 'WI', '<SPAN CLASS="PROPERTY-TENNESSEE</SPAN>' : 'TN',
                '<SPAN CLASS="PROPERTY-MINNESOTA</SPAN>' : 'MN', '<SPAN CLASS="PROPERTY-CALIFORNIA</SPAN>' : 'CA',
                '<SPAN CLASS="PROPERTY-TEXAS</SPAN>' : 'TX'}

In [76]:
merged_hotels['state'] = merged_hotels['state'].replace(statereplace2)


In [77]:
merged_hotels['state'].value_counts()

TX                   2300
CA                   1653
FL                   1495
GA                    913
NC                    783
NY                    753
VA                    714
OH                    698
PA                    686
IL                    681
TN                    603
MI                    548
CO                    514
IN                    487
SC                    479
AZ                    457
WI                    426
MO                    402
MN                    399
AL                    399
OK                    375
LA                    372
KY                    349
MD                    347
IA                    343
WA                    336
NJ                    322
MA                    304
AR                    280
KS                    257
MS                    255
UT                    250
NM                    229
OR                    211
NE                    174
NV                    162
CT                    151
SD                    143
WV          

In [78]:
merged_hotels.isnull().sum()

rewards          0
brand          137
name             0
rating          92
description     27
street          74
city            74
state           78
postal          74
url              0
dtype: int64

In [79]:
merged_hotels2 = merged_hotels[~merged_hotels['state'].isnull()]
merged_hotels2.shape

(21367, 10)

In [80]:
merged_hotels2.isnull().sum()

rewards          0
brand          137
name             0
rating          92
description     27
street           0
city             0
state            0
postal           0
url              0
dtype: int64

In [81]:
merged_hotels2[merged_hotels2['brand'].isnull()]

,rewards,brand,name,rating,description,street,city,state,postal,url
9636,Marriott,NaN,Aloft Birmingham Soho Square,4.2,[ Surrounded by the chic boutiques and renowne...,1903 29th Avenue South,Homewood,AL,35209,https://www.marriott.com/BHMAL
9637,Marriott,NaN,Aloft Glendale at Westgate,4.4,[ Change it up at Aloft® Hotels Glendale at We...,6920 North 93rd Avenue,Glendale,AZ,85305,https://www.marriott.com/PHXAW
9638,Marriott,NaN,Aloft Phoenix-Airport,4.1,[ Enjoy yourself at Aloft Phoenix-Airport. Our...,4450 East Washington,Phoenix,AZ,85034,https://www.marriott.com/PHXXL
9639,Marriott,NaN,Aloft Scottsdale,4.0,[ Experience an exceptional stay in a desirabl...,4415 North Civic Center Plaza,Scottsdale,AZ,85251,https://www.marriott.com/PHXSD
9640,Marriott,NaN,Aloft Tempe,3.8,[ Just a mile from Arizona State University (A...,951 East Playa Del Norte Drive,Tempe,AZ,85281,https://www.marriott.com/PHXAL
9641,Marriott,NaN,Aloft Tucson University,4.1,[ Trendy style blends with a convenient locati...,1900 East Speedway Boulevard,Tucson,AZ,85719,https://www.marriott.com/TUSAL
9642,Marriott,NaN,Aloft Rogers-Bentonville,4.1,[ Surrounded by an endless array of attraction...,1103 South 52nd Street,Rogers,AR,72758,https://www.marriott.com/ROGAL
9643,Marriott,NaN,Aloft Cupertino,4.0,[ A celebration of style and comfort awaits yo...,10165 North De Anza Boulevard,Cupertino,CA,95014,https://www.marriott.com/SJCUP
9644,Marriott,NaN,Aloft Dublin-Pleasanton,4.4,[ Technology and taste collide at the brand-ne...,4075 Grafton Street,Dublin,CA,94568,https://www.marriott.com/OAKAL
9645,Marriott,NaN,Aloft El Segundo - Los Angeles Airport,4.0,[ Enjoy yourself at Aloft El Segundo - Los Ang...,475 North Pacific Coast Highway,El Segundo,CA,90245,https://www.marriott.com/LAXLA


In [82]:
for index, row in merged_hotels2.iterrows():
    if 'Aloft' in row['name']:
        merged_hotels2['brand'][index] = 'Aloft'

/Users/chrisjohnson/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [83]:
merged_hotels2['rating'] = merged_hotels2['rating'].str.strip()

/Users/chrisjohnson/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [84]:
merged_hotels2 = merged_hotels2[~merged_hotels2['rating'].isnull()]

In [85]:
ratingreplace = {'0' : 'New Hotel', '4.5 Stars':'4.5', 'https://www.tripadvisor.com/img/cdsi/img2/ratings/traveler/4.0-MCID-5.svg' : '4',
                '4.0 Stars' : '4', 'https://www.tripadvisor.com/img/cdsi/img2/ratings/traveler/4.5-MCID-5.svg' : '4.5',
                'https://www.tripadvisor.com/img/cdsi/img2/ratings/traveler/3.5-MCID-5.svg' : '3.5', '5.0 Stars' : '5',
                'https://www.tripadvisor.com/img/cdsi/img2/ratings/traveler/3.0-MCID-5.svg' : '3', '3.5 Stars' : '3.5',
                'https://www.tripadvisor.com/img/cdsi/img2/ratings/traveler/5.0-MCID-5.svg' : '5', 
                'https://www.tripadvisor.com/img/cdsi/img2/ratings/traveler/2.5-MCID-5.svg' : '2.5', '2.0 Stars' : '2',
                 '3.0 Stars' : '3', 'https://www.tripadvisor.com/img/cdsi/img2/ratings/traveler/2.0-MCID-5.svg' : '2',
                 'https://www.tripadvisor.com/img/cdsi/img2/ratings/traveler/0.0-MCID-5.svg' : '0', 'nager' : 'New Hotel'
                }
merged_hotels2['rating'] = merged_hotels2['rating'].replace(ratingreplace)


In [86]:
merged_hotels2['rating'].value_counts()

4.5          5517
4            2162
3.5          2010
4.0          1649
4.6          1598
3.0          1345
4.4          1305
4.7          1235
New Hotel     773
4.3           767
2.5           659
4.8           615
4.2           493
4.1           302
5             188
2.0           166
3.9           119
4.9           116
3.8            56
3              51
5.0            44
3.7            34
3.6            19
1.5            13
2              11
3.4             5
0.0             5
3.3             4
1.0             3
1               2
4,5             2
4,6             1
2.8             1
0               1
2.9             1
1.8             1
4,7             1
3.1             1
Name: rating, dtype: int64

In [87]:
newrating = []
for x in merged_hotels2['rating']:
    try:
        x = float(x)
        if x == 5.0:
            z = 5.0
            newrating.append(z)
        elif 4.5 <= x < 5.0:
            z = 4.5
            newrating.append(z)
        elif 4.0 <= x < 4.5:
            z = 4.0
            newrating.append(z)
        elif 3.5 <= x < 4.0:
            z = 3.5
            newrating.append(z)
        elif 3.0 <= x < 3.5:
            z = 3.0
            newrating.append(z)
        elif 2.5 <= x < 3.0:
            z = 2.5
            newrating.append(z)
        elif 2.0 <= x < 2.5:
            z = 2.0
            newrating.append(z)
        elif 1.5 <= x < 2.0:
            z = 1.5
            newrating.append(z)
        elif 1.0 <= x < 1.5:
            z = 1.0
            newrating.append(z)
        elif .5 <= x < 1.0:
            z = .5
            newrating.append(z)
        elif 0.0 <= x < .5:
            z = 0.0
            newrating.append(z)
    except:
        newrating.append(x)

In [88]:
merged_hotels2['rating'] = newrating

In [89]:
replaceratings = {'4,6' : 4.5, '4,5' : 4.5, '4,7' : 4.5}

In [90]:
merged_hotels2['rating'] = merged_hotels2['rating'].replace(replaceratings)

In [91]:
merged_hotels2['rating'].value_counts()

4.5          9085
4.0          6678
3.5          2238
3.0          1406
New Hotel     773
2.5           661
5.0           232
2.0           177
1.5            14
0.0             6
1.0             5
Name: rating, dtype: int64

In [92]:
merged_hotels2.dtypes

rewards        object
brand          object
name           object
rating         object
description    object
street         object
city           object
state          object
postal         object
url            object
dtype: object

In [93]:
print(merged_hotels2['rewards'].value_counts())
print(merged_hotels2['rewards'].value_counts().sum())

Wyndham     6073
Marriott    5191
Hilton      4890
IHG         3875
Hyatt        713
Radisson     533
Name: rewards, dtype: int64
21275


In [94]:
print(merged_hotels2['brand'].value_counts())
print(merged_hotels2['brand'].value_counts().sum())

Holiday Inn Express       2257
Hampton Inn               2165
Super 8                   1474
Days Inn                  1375
Fairfield                 1050
Courtyard                 1040
La Quinta                  888
Residence Inn              841
Hilton Garden Inn          675
Holiday Inn                618
Baymont                    501
Springhill                 500
Homewood Suites            471
Home2                      455
TownPlace                  448
Country Inn & Suites       446
Candlewood                 419
Marriott                   378
Doubletree                 350
Travelodge                 335
Hyatt Place                323
Ramada                     315
Microtel                   291
Staybridge                 289
Hilton                     242
Embassy Suites             236
AmericInn                  199
Tru                        190
Sheraton                   175
Wingate                    163
Howard Johnson             159
Aloft                      137
FourPoin

In [95]:
print(merged_hotels2['rating'].value_counts())
print(merged_hotels2['rating'].value_counts().sum())

4.5          9085
4.0          6678
3.5          2238
3.0          1406
New Hotel     773
2.5           661
5.0           232
2.0           177
1.5            14
0.0             6
1.0             5
Name: rating, dtype: int64
21275


In [96]:
print(merged_hotels2['state'].value_counts())
print(merged_hotels2['state'].value_counts().sum())

TX                   2298
CA                   1639
FL                   1485
GA                    911
NC                    780
NY                    749
VA                    713
OH                    697
PA                    683
IL                    677
TN                    602
MI                    548
CO                    512
IN                    487
SC                    479
AZ                    456
WI                    425
MO                    402
MN                    399
AL                    399
OK                    375
LA                    371
KY                    349
MD                    346
IA                    343
WA                    330
NJ                    322
MA                    301
AR                    280
KS                    257
MS                    255
UT                    249
NM                    229
OR                    208
NE                    173
NV                    162
CT                    151
SD                    143
WV          

In [97]:
intercontinental = merged_hotels2[merged_hotels2['brand'].str.contains('Inter')]

In [98]:
intercontinental = intercontinental.rename(columns={'city' : 'state', 'state': 'city'})
intercontinental = intercontinental[['rewards', 'brand', 'name', 'rating', 'description', 'street', 'city', 'state', 'postal', 'url']]

In [99]:
intercontinental['state'] = intercontinental['state'].str.upper()

In [100]:
intercontinental['city'] = intercontinental['city'].str.title()

In [101]:
intercontinental.head()

,rewards,brand,name,rating,description,street,city,state,postal,url
9548,IHG,InterContinental,InterContinental Hotels Los Angeles Downtown,4,\nRevel in glamour and exhilaration that defin...,900 Wilshire Boulevard,Los Angeles,CALIFORNIA,90017,https://www.ihg.com/intercontinental/hotels/us...
9549,IHG,InterContinental,InterContinental Hotels San Francisco,4,\nImmerse yourself in the heart of the city a...,888 Howard Street,San Francisco,CALIFORNIA,94103,https://www.ihg.com/intercontinental/hotels/us...
9550,IHG,InterContinental,InterContinental Hotels Mark Hopkins San Franc...,4,\nDiscover San Francisco's historic charm at t...,999 California Street,San Francisco,CALIFORNIA,94108,https://www.ihg.com/intercontinental/hotels/us...
9551,IHG,InterContinental,InterContinental Hotels Los Angeles Century City,4.5,\nEnjoy unrivaled hospitality at the InterCont...,2151 Avenue Of The Stars,Los Angeles,CALIFORNIA,90067,https://www.ihg.com/intercontinental/hotels/us...
9552,IHG,InterContinental,InterContinental Hotels San Diego,4.5,\nExuding the liveliness of the city with a so...,901 Bayfront Court,San Diego,CALIFORNIA,92101,https://www.ihg.com/intercontinental/hotels/us...


In [102]:
merged_hotels3 = merged_hotels2[~merged_hotels2['brand'].str.contains('Inter')]

In [103]:
merged_hotels3.shape

(21250, 10)

In [104]:
merged_hotels3 = pd.concat([merged_hotels3, intercontinental], ignore_index=True)

In [105]:
merged_hotels3.shape

(21275, 10)

In [106]:
print(merged_hotels3['state'].value_counts())
print(merged_hotels3['state'].value_counts().sum())

TX                      2298
CA                      1639
FL                      1485
GA                       911
NC                       780
NY                       747
VA                       713
OH                       697
PA                       683
IL                       677
TN                       602
MI                       548
CO                       512
IN                       487
SC                       479
AZ                       456
WI                       425
MO                       402
MN                       399
AL                       399
OK                       375
LA                       371
KY                       349
MD                       346
IA                       343
WA                       328
NJ                       322
MA                       301
AR                       280
KS                       257
MS                       255
UT                       249
NM                       229
OR                       208
NE            

In [107]:
merged_hotels3[merged_hotels3['state'].str.contains('CHAR')]

,rewards,brand,name,rating,description,street,city,state,postal,url
14959,Radisson,Radisson,Radisson Hotel Charlotte Airport,4.5,Unpack your bags after a 10-minute shuttle ri...,212 W Woodlawn Road,#C,CHAR,lotte,https://www.radissonhotels.com/en-us/hotels/ra...


In [108]:
statereplace3 = {'LA C' : 'WI', '200 Harborview Plaza' : 'La Crosse', 'rosse' : '54601', 'CHAR' : 'NC', 'lotte' : '28217', '#C' : 'Charlotte', 'ILLINOIS61036' : 'IL', 'PENNSYLVANIA17011' : 'PA', 'NEW YORK14830' : 'NY'}

In [109]:
merged_hotels3['state'] = merged_hotels3['state'].replace(statereplace)

In [110]:
merged_hotels3 = merged_hotels3.replace(statereplace3)

In [111]:
merged_hotels3['city'] = merged_hotels3['city'].str.strip()

In [112]:
merged_hotels3['brand'].unique()

array(['Hampton Inn', 'Tru', 'Hilton Garden Inn', 'Home2', 'Doubletree',
       'Homewood Suites', 'Curio', 'Hilton', 'Embassy Suites', 'Canopy',
       'Hilton Grand Vacations', 'Conrad', 'Waldorf Astoria', 'Tapestry',
       'Hyatt Place', 'Hyatt Regency', 'Houte House', 'Andaz', 'MiRaval',
       'Destination or Unbound', 'Joie De Vivre', 'Grand Hyatt',
       'Hyatt Centric', 'Hyatt', 'Park Hyatt', 'Thompson',
       'Holiday Inn Express', 'Holiday Inn', 'Indigo', 'Candlewood',
       'Staybridge', 'EVEN', 'Crowne Plaza', 'Kimpton', 'AC Hotel',
       'Aloft', 'Courtyard', 'Delta Hotels', 'Fairfield', 'FourPoints',
       'JW Marriott', 'Le Meredian', 'Luxury Collection', 'Marriott',
       'Moxy', 'Renaissance', 'Residence Inn', 'Sheraton', 'Springhill',
       'St. Regis', 'TownPlace', 'Tribute', 'Westin', 'W Hotel',
       'Country Inn & Suites', 'Radisson', 'Park Inn', 'Microtel',
       'Days Inn', 'Super 8', 'La Quinta', 'Ramada', 'Baymont',
       'Howard Johnson', 'Wyndham 

In [113]:
merged_hotels3['brand'].value_counts()

Holiday Inn Express       2257
Hampton Inn               2165
Super 8                   1474
Days Inn                  1375
Fairfield                 1050
Courtyard                 1040
La Quinta                  888
Residence Inn              841
Hilton Garden Inn          675
Holiday Inn                618
Baymont                    501
Springhill                 500
Homewood Suites            471
Home2                      455
TownPlace                  448
Country Inn & Suites       446
Candlewood                 419
Marriott                   378
Doubletree                 350
Travelodge                 335
Hyatt Place                323
Ramada                     315
Microtel                   291
Staybridge                 289
Hilton                     242
Embassy Suites             236
AmericInn                  199
Tru                        190
Sheraton                   175
Wingate                    163
Howard Johnson             159
Aloft                      137
FourPoin

In [114]:
brandlist = ['Hampton Inn', 'Tru', 'Hilton Garden Inn', 'Home2', 'Doubletree',
       'Homewood Suites', 'Curio', 'Hilton', 'Embassy Suites', 'Canopy',
       'Hilton Grand Vacations', 'Conrad', 'Waldorf Astoria', 'Tapestry',
       'Hyatt Place', 'Hyatt Regency', 'Houte House', 'Andaz', 'MiRaval',
       'Destination or Unbound', 'Joie De Vivre', 'Grand Hyatt',
       'Hyatt Centric', 'Hyatt', 'Park Hyatt', 'Thompson',
       'Holiday Inn Express', 'Holiday Inn', 'Indigo', 'Candlewood',
       'Staybridge', 'EVEN', 'Crowne Plaza', 'Kimpton', 'AC Hotel',
       'Aloft', 'Courtyard', 'Delta Hotels', 'Fairfield', 'FourPoints',
       'JW Marriott', 'Le Meredian', 'Luxury Collection', 'Marriott',
       'Moxy', 'Renaissance', 'Residence Inn', 'Sheraton', 'Springhill',
       'St. Regis', 'TownPlace', 'Tribute', 'Westin', 'W Hotel',
       'Country Inn & Suites', 'Radisson', 'Park Inn', 'Microtel',
       'Days Inn', 'Super 8', 'La Quinta', 'Ramada', 'Baymont',
       'Howard Johnson', 'Wyndham Garden', 'Wingate', 'Travelodge',
       'Trademark', 'Hawthorn Suites', 'Wyndham Vacations', 'AmericInn',
       'TRYP', 'Club Wyndham', 'Wyndham Grand', 'Wyndham', 'Paris',
       'InterContinental']

In [115]:
brandlist.sort()

In [116]:
brandlist

['AC Hotel',
 'Aloft',
 'AmericInn',
 'Andaz',
 'Baymont',
 'Candlewood',
 'Canopy',
 'Club Wyndham',
 'Conrad',
 'Country Inn & Suites',
 'Courtyard',
 'Crowne Plaza',
 'Curio',
 'Days Inn',
 'Delta Hotels',
 'Destination or Unbound',
 'Doubletree',
 'EVEN',
 'Embassy Suites',
 'Fairfield',
 'FourPoints',
 'Grand Hyatt',
 'Hampton Inn',
 'Hawthorn Suites',
 'Hilton',
 'Hilton Garden Inn',
 'Hilton Grand Vacations',
 'Holiday Inn',
 'Holiday Inn Express',
 'Home2',
 'Homewood Suites',
 'Houte House',
 'Howard Johnson',
 'Hyatt',
 'Hyatt Centric',
 'Hyatt Place',
 'Hyatt Regency',
 'Indigo',
 'InterContinental',
 'JW Marriott',
 'Joie De Vivre',
 'Kimpton',
 'La Quinta',
 'Le Meredian',
 'Luxury Collection',
 'Marriott',
 'MiRaval',
 'Microtel',
 'Moxy',
 'Paris',
 'Park Hyatt',
 'Park Inn',
 'Radisson',
 'Ramada',
 'Renaissance',
 'Residence Inn',
 'Sheraton',
 'Springhill',
 'St. Regis',
 'Staybridge',
 'Super 8',
 'TRYP',
 'Tapestry',
 'Thompson',
 'TownPlace',
 'Trademark',
 'Travel

In [117]:
Holiday Inn Express       2257
Hampton Inn               2165
Super 8                   1474
Days Inn                  1375
Fairfield                 1050
Courtyard                 1040
La Quinta                  888
Residence Inn              841
Hilton Garden Inn          675
Holiday Inn                593
Baymont                    501
Springhill                 500
Homewood Suites            471
Home2                      455
TownPlace                  448
Country Inn & Suites       446
Candlewood                 419
Marriott                   378
Doubletree                 350
Travelodge                 335
Hyatt Place                323
Ramada                     315
Microtel                   291
Staybridge                 289
Hilton                     242
Embassy Suites             236
AmericInn                  199
Tru                        190
Sheraton                   175
Wingate                    163
Howard Johnson             159
Aloft                      137
FourPoints                 131
Westin                     124
Crowne Plaza               114
Hawthorn Suites            101
Houte House                100
Hyatt Regency               97
Destination or Unbound      97
AC Hotel                    84
Renaissance                 81
Wyndham Vacations           81
Radisson                    81
Indigo                      69
Club Wyndham                67
Wyndham Garden              67
Kimpton                     66
Curio                       58
Delta Hotels                42
Trademark                   38
JW Marriott                 32
W Hotel                     28
Tribute                     28
InterContinental            25
Moxy                        23
Hyatt Centric               22
Le Meredian                 21
Hilton Grand Vacations      21
Hyatt                       20
Luxury Collection           19
Canopy                      19
EVEN                        18
Joie De Vivre               16
Grand Hyatt                 13
St. Regis                    9
TRYP                         9
Andaz                        9
Wyndham Grand                8
Thompson                     7
Park Hyatt                   6
Park Inn                     6
Conrad                       6
MiRaval                      3
Wyndham                      1
Tapestry                     1
Waldorf Astoria              1
Paris                        1

SyntaxError: invalid syntax (<ipython-input-117-c117f11d2b4d>, line 1)

In [118]:
classnumber = {'AC Hotel':3,
 'Aloft':3,
 'AmericInn':5,
 'Andaz':1,
 'Baymont':5,
 'Candlewood':5,
 'Canopy':2,
 'Club Wyndham':2,
 'Conrad':1,
 'Country Inn & Suites':4,
 'Courtyard':3,
 'Crowne Plaza':3,
 'Curio':2,
 'Days Inn':6,
 'Delta Hotels':3,
 'Destination or Unbound':1,
 'Doubletree':3,
 'EVEN':3,
 'Embassy Suites':2,
 'Fairfield':4,
 'FourPoints':3,
 'Grand Hyatt':1,
 'Hampton Inn':4,
 'Hawthorn Suites':5,
 'Hilton':2,
 'Hilton Garden Inn':3,
 'Hilton Grand Vacations':2,
 'Holiday Inn':4,
 'Holiday Inn Express':4,
 'Home2':4,
 'Homewood Suites':3,
 'Houte House':3,
 'Howard Johnson':6,
 'Hyatt':2,
 'Hyatt Centric':2,
 'Hyatt Place':3,
 'Hyatt Regency':2,
 'Indigo':2,
 'InterContinental':1,
 'JW Marriott':1,
 'Joie De Vivre':2,
 'Kimpton':2,
 'La Quinta':4,
 'Le Meredian':2,
 'Luxury Collection':1,
 'Marriott':2,
 'MiRaval':1,
 'Microtel':6,
 'Moxy':4,
 'Paris':2,
 'Park Hyatt':1,
 'Park Inn':4,
 'Radisson':3,
 'Ramada':5,
 'Renaissance':2,
 'Residence Inn':3,
 'Sheraton':2,
 'Springhill':3,
 'St. Regis':1,
 'Staybridge':3,
 'Super 8':6,
 'TRYP':4,
 'Tapestry':3,
 'Thompson':1,
 'TownPlace':4,
 'Trademark':4,
 'Travelodge':6,
 'Tribute':2,
 'Tru':5,
 'W Hotel':1,
 'Waldorf Astoria':1,
 'Westin':2,
 'Wingate':5,
 'Wyndham':3,
 'Wyndham Garden':4,
 'Wyndham Grand':2,
 'Wyndham Vacations':3}

In [119]:
merged_hotels3['class_id'] = merged_hotels3['brand']

In [120]:
merged_hotels3['class_id'] = merged_hotels3['class_id'].replace(classnumber)

In [121]:
merged_hotels3.head()

,rewards,brand,name,rating,description,street,city,state,postal,url,class_id
0,Hilton,Hampton Inn,Hampton Inn Alexander City,4,"We’re right off Highway 280, 25 minutes away f...",1551 Elkahatchee Road,Alexander City,AL,35010,https://hamptoninn3.hilton.com/en/hotels/alaba...,4
1,Hilton,Hampton Inn,Hampton Inn Wetumpka,5,"We’re on the banks of the Coosa River, a short...",350 South Main Street,Wetumpka,AL,36092,https://hamptoninn3.hilton.com/en/hotels/alaba...,4
2,Hilton,Hampton Inn,Hampton Inn Auburn,4,"We're off I-85, under 10 minutes from Chewacla...",2430 S. College St.,Auburn,AL,36832,https://hamptoninn3.hilton.com/en/hotels/alaba...,4
3,Hilton,Tru,Tru by Hilton Auburn,4,"Make this Tru by Hilton your own. We’ll help, ...",2411 W Pace Blvd,Auburn,AL,36830,https://tru3.hilton.com/en/hotels/alabama/tru-...,5
4,Hilton,Hilton Garden Inn,Hilton Garden Inn Auburn/Opelika,4,"Our hotel is just off I-85, minutes away from ...",2555 Hilton Garden Drive,Auburn,AL,36830,https://hiltongardeninn3.hilton.com/en/hotels/...,3


In [122]:
classcategory = {'AC Hotel':'Lifestyle, Boutique',
 'Aloft':'Lifestyle, Boutique',
 'AmericInn':'Budget',
 'Andaz':'Lifestyle, Boutique',
 'Baymont':'Budget',
 'Candlewood':'Extended-Stay, All-Suite',
 'Canopy':'Lifestyle, Boutique',
 'Club Wyndham':'Vacation Club, Condos, Villas',
 'Conrad':'Luxury',
 'Country Inn & Suites':'Limited-Service Mid-Scale',
 'Courtyard':'Full-Service Entry',
 'Crowne Plaza':'Full-Service Mid-Scale',
 'Curio':'Lifestyle, Boutique',
 'Days Inn':'Budget',
 'Delta Hotels':'Full-Service Entry',
 'Destination or Unbound':'Lifestyle, Boutique',
 'Doubletree':'Full-Service Mid-Scale',
 'EVEN':'Lifestyle, Boutique',
 'Embassy Suites':'Extended-Stay, All-Suite',
 'Fairfield':'Limited-Service Mid-Scale',
 'FourPoints':'Full-Service Entry',
 'Grand Hyatt':'Luxury',
 'Hampton Inn':'Limited-Service Mid-Scale',
 'Hawthorn Suites':'Extended-Stay, All-Suite',
 'Hilton':'Full-Service Upper-Scale',
 'Hilton Garden Inn':'Full-Service Entry',
 'Hilton Grand Vacations':'Vacation Club, Condos, Villas',
 'Holiday Inn':'Full-Service Mid-Scale',
 'Holiday Inn Express':'Limited-Service Mid-Scale',
 'Home2':'Extended-Stay, All-Suite',
 'Homewood Suites':'Extended-Stay, All-Suite',
 'Houte House':'Extended-Stay, All-Suite',
 'Howard Johnson':'Budget',
 'Hyatt':'Lifestyle, Boutique',
 'Hyatt Centric':'Lifestyle, Boutique',
 'Hyatt Place':'Limited-Service Mid-Scale',
 'Hyatt Regency':'Full-Service Upper-Scale',
 'Indigo':'Lifestyle, Boutique',
 'InterContinental':'Luxury',
 'JW Marriott':'Luxury',
 'Joie De Vivre':'Lifestyle, Boutique',
 'Kimpton':'Lifestyle, Boutique',
 'La Quinta':'Limited-Service Mid-Scale',
 'Le Meredian':'Lifestyle, Boutique',
 'Luxury Collection':'Luxury',
 'Marriott':'Full-Service Upper-Scale',
 'MiRaval':'Luxury',
 'Microtel':'Budget',
 'Moxy':'Lifestyle, Boutique',
 'Paris':'Full-Service Upper-Scale',
 'Park Hyatt':'Luxury',
 'Park Inn':'Limited-Service Mid-Scale',
 'Radisson':'Full-Service Entry',
 'Ramada':'Budget',
 'Renaissance':'Full-Service Upper-Scale',
 'Residence Inn':'Extended-Stay, All-Suite',
 'Sheraton':'Full-Service Upper-Scale',
 'Springhill':'Extended-Stay, All-Suite',
 'St. Regis':'Luxury',
 'Staybridge':'Extended-Stay, All-Suite',
 'Super 8':'Budget',
 'TRYP':'Lifestyle, Boutique',
 'Tapestry':'Lifestyle, Boutique',
 'Thompson':'Lifestyle, Boutique',
 'TownPlace':'Extended-Stay, All-Suite',
 'Trademark':'Lifestyle, Boutique',
 'Travelodge':'Budget',
 'Tribute':'Lifestyle, Boutique',
 'Tru':'Budget',
 'W Hotel':'Luxury',
 'Waldorf Astoria':'Luxury',
 'Westin':'Full-Service Upper-Scale',
 'Wingate':'Budget',
 'Wyndham':'Full-Service Entry',
 'Wyndham Garden':'Limited-Service Mid-Scale',
 'Wyndham Grand':'Full-Service Mid-Scale',
 'Wyndham Vacations':'Vacation Club, Condos, Villas'}

In [123]:
merged_hotels3['category'] = merged_hotels3['brand']

In [124]:
merged_hotels3['category'] = merged_hotels3['category'].replace(classcategory)

In [125]:
merged_hotels3['category'].value_counts()

Limited-Service Mid-Scale        7202
Budget                           5002
Extended-Stay, All-Suite         3860
Full-Service Entry               1970
Full-Service Upper-Scale         1098
Full-Service Mid-Scale           1090
Lifestyle, Boutique               742
Vacation Club, Condos, Villas     169
Luxury                            142
Name: category, dtype: int64

In [126]:
merged_hotels3['class_id'].value_counts()

4    8470
3    5279
6    3634
5    1888
2    1749
1     255
Name: class_id, dtype: int64

In [129]:
merged_hotels3 = merged_hotels3[~merged_hotels3['description'].isnull()]

In [132]:
merged_hotels3.dtypes

rewards        object
brand          object
name           object
rating         object
description    object
street         object
city           object
state          object
postal         object
url            object
class_id        int64
category       object
dtype: object

In [133]:
merged_hotels3.to_csv('all_hotels_cleaned.csv')